In [ ]:
# !pip install catboost

In [1]:
from pyforest import *
import seaborn as sns

from collections import Counter
from imblearn.under_sampling import TomekLinks
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
df = pd.read_csv('heart.csv')
df

<IPython.core.display.Javascript object>

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [3]:
# Checking to see how the target varible is split up
def value_counts(col):
    print('Raw counts: \n')
    print(col.value_counts())
    print('-----------------------------------')
    print('Normalized counts: \n')
    print(col.value_counts(normalize=True))
value_counts(df['output'])

Raw counts: 

1    165
0    138
Name: output, dtype: int64
-----------------------------------
Normalized counts: 

1    0.544554
0    0.455446
Name: output, dtype: float64


## Train-Test Split:

In [4]:
from sklearn.model_selection import train_test_split

#spliting the df
y = df['output']
x = df.drop('output', axis=1)
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=1)

In [5]:
tl = TomekLinks()
X_res, y_res = tl.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

Resampled dataset shape Counter({1: 111, 0: 108})


In [7]:
X_res, y_res = tl.fit_resample(X_res, y_res)
print('Resampled dataset shape %s' % Counter(y_res))

Resampled dataset shape Counter({0: 108, 1: 104})


## Scaler

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_res)
scaled_X_train = pd.DataFrame(scaler.fit_transform(X_res), index=X_res.index, columns=X_res.columns)
scaled_X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## XGBoost:

In [9]:
#Need to turn X_train and X_test into a DMatrix
D_train = xgb.DMatrix(scaled_X_train, label=y_res)
D_test = xgb.DMatrix(scaled_X_test, label=y_test)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
y_train = y_res
X_train = X_res

In [14]:
param = {}

In [15]:
#Creating the classifier
model = xgb.train(param, D_train, num_boost_round = 100)

#fit the model
# model.fit(scaled_X_train, y_train)

# Predict on the test set
test_preds = model.predict(D_test)
train_preds = model.predict(D_train)

<IPython.core.display.Javascript object>

In [16]:
#Checking the test and train scores
best_preds = np.asarray([np.argmax(line) for line in test_preds])
best_preds_train = np.asarray([np.argmax(line) for line in train_preds])


print('Training Precision: ', precision_score(y_train, best_preds_train, average='macro'))
print('Testing Precision: ', precision_score(y_test, best_preds, average='macro'))
print('Training Recall: ', recall_score(y_train, best_preds_train, average='macro'))
print('Testing Recall: ', recall_score(y_test, best_preds, average='macro'))
print('Training Accuracy: ', accuracy_score(y_train, best_preds_train))
print('Testing Accuracy: ', accuracy_score(y_test, best_preds))
print('Training F1-Score: ', f1_score(y_train, best_preds_train, average='macro'))
print('Testing F1-Score: ', f1_score(y_test, best_preds, average='macro'))

cm = confusion_matrix(y_test, (best_preds>0.5))
print(('The Confusion Matrix is: '),'\n', cm)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Training Precision:  0.25471698113207547
Testing Precision:  0.2459016393442623
Training Recall:  0.5
Testing Recall:  0.5
Training Accuracy:  0.5094339622641509
Testing Accuracy:  0.4918032786885246
Training F1-Score:  0.33749999999999997
Testing F1-Score:  0.32967032967032966
The Confusion Matrix is:  
 [[30  0]
 [31  0]]


## Gridsearch

In [17]:
clf = xgb.XGBClassifier()
parameters = {
    'eta': [0.005, 0.01, 0.02, 0.3],  
    'max_depth': [2, 3, 4, 5, 6],  
    'objective': ['multi:softprob', 'multi:softmax'],
    'num_class': [2, 3, 4, 5]} 

grid = GridSearchCV(clf,
                    parameters, n_jobs=4,
                    scoring="recall",
                    cv=5,
                    verbose=3)
#                     return_train_score=True)

grid.fit(scaled_X_train, y_train)

<IPython.core.display.Javascript object>

Fitting 5 folds for each of 160 candidates, totalling 800 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    1.9s
[Parallel(n_jobs=4)]: Done 380 tasks      | elapsed:    8.7s
[Parallel(n_jobs=4)]: Done 793 out of 800 | elapsed:   16.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done 800 out of 800 | elapsed:   16.1s finished


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None,

In [18]:
grid.best_params_

{'eta': 0.02, 'max_depth': 4, 'num_class': 5, 'objective': 'multi:softprob'}

In [22]:
params_final = {'eta': 0.02, 'max_depth': 4, 'num_class': 5, 'objective': 'multi:softprob'}

In [23]:
model2 = xgb.train(params_final, D_train, num_boost_round = 100)

# Fit the classifier
# model2.fit(scaled_X_train, y_train)

# Predict on the test set
test_preds = model.predict(D_test)
train_preds = model.predict(D_train)

<IPython.core.display.Javascript object>

In [24]:
#Checking the test and train scores
best_preds = np.asarray([np.argmax(line) for line in test_preds])
best_preds_train = np.asarray([np.argmax(line) for line in train_preds])


print('Training Precision: ', precision_score(y_train, best_preds_train, average='macro'))
print('Testing Precision: ', precision_score(y_test, best_preds, average='macro'))
print('Training Recall: ', recall_score(y_train, best_preds_train, average='macro'))
print('Testing Recall: ', recall_score(y_test, best_preds, average='macro'))
print('Training Accuracy: ', accuracy_score(y_train, best_preds_train))
print('Testing Accuracy: ', accuracy_score(y_test, best_preds))
print('Training F1-Score: ', f1_score(y_train, best_preds_train, average='macro'))
print('Testing F1-Score: ', f1_score(y_test, best_preds, average='macro'))

cm = confusion_matrix(y_test, (best_preds>0.5))
print(('The Confusion Matrix is: '),'\n', cm)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Training Precision:  0.25471698113207547
Testing Precision:  0.2459016393442623
Training Recall:  0.5
Testing Recall:  0.5
Training Accuracy:  0.5094339622641509
Testing Accuracy:  0.4918032786885246
Training F1-Score:  0.33749999999999997
Testing F1-Score:  0.32967032967032966
The Confusion Matrix is:  
 [[30  0]
 [31  0]]
